# FASTGenomics Scanpy + R Analysis

<font color="red">You might want to describe your analysis briefly here, if you are planning to share it.</font>

In [ ]:
# Place all your Python imports here.
import logging
import fgread
import scanpy as sc

# do not delete these imports as they are required for R support
import rpy2.rinterface_lib.callbacks
from rpy2.robjects import pandas2ri
import anndata2ri
%load_ext rpy2.ipython

In [ ]:
%%R
# Place all your R library imports here
suppressPackageStartupMessages({
    library(scran)
})

In [ ]:
# Place all your parameter values here.

sc.settings.verbosity = 1                                    # scanpy verbosity: errors (0), warnings (1), info (2), hints (3)
rpy2.rinterface_lib.callbacks.logger.setLevel(logging.ERROR) # Ignore R warning messages

# Automatically convert rpy2 outputs to pandas dataframes
pandas2ri.activate()
anndata2ri.activate()

## Raw Data
First, the raw dataset(s) will be read into an AnnData object(s).

<font color="red">You can describe your data here using markdown or delete this text.</font>

In [ ]:
# Load metadata of each dataset
dsets = fgread.get_datasets()
dsets

In [ ]:
# Read all datasets into a dictionary of AnnData objects
adata_dct = fgread.read_datasets() # If you have multiple datasets they will be all loaded into RAM.

# You can, for example, iterate over your datasets and print basic information for each
for i,adata in adata_dct.items():
    print(i, adata)

In [ ]:
# You can also select a single dataset by its ID like below. This will return a single AnnData object and save you some RAM if you don't neet all datasets right away
# adata = fgread.read_datasets(dsets[1])

## Preprocessing
If this is your first analysis, you can find examples [here](https://beta.fastgenomics.org/webclient/ui/#/analyses/detail-analysis-c4a597f0842941aa9d73072e473de91d) or [here](https://beta.fastgenomics.org/webclient/ui/#/analyses/detail-analysis-37e3664431a34d4a9bb6b9f725036f48).

<font color="red">You can describe your preprocessing here or delete this text.</font>

In [ ]:
# This is an example of how to prepare AnnData matrices for processing with R
data = adata_dct[1].X.T.todense()

In [ ]:
%%R -i data -o clusters
# You can use R code in cells with specified inputs and outputs
clusters <- quickCluster(data)

In [ ]:
# The outputs are then available in Python
clusters